# 사진 분류 인공지능 구현하기

In [1]:
import torch
import numpy as np
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.models as models
import time
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

In [2]:
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

True
cuda


In [3]:
# 사전학습된 ResNet 불러오기
resnet = models.resnet50(pretrained=True).to(device)

# 모델 summary 출력
from torchsummary import summary
summary(resnet, input_size=(3, 224, 224), device=device.type)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 85.2MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
# 이미지 불러오기
from google.colab import files
img = files.upload()

In [ ]:
from PIL import Image
img = Image.open('/content/cat.jpg').convert('RGB')
img

In [ ]:
# 데이터 전처리
from torchvision import transforms
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

])
img_preprocessed = preprocess(img)
batch_img_tensor = torch.unsqueeze(img_preprocessed, 0)

In [ ]:
# 테스트용 이미지 결과 출력
resnet.eval()
out = resnet(batch_img_tensor.to(device))

In [ ]:
# imagenet class 불러오기 (imagenet_classes.txt)
# txt파일을 한번 열어 보세요!
from google.colab import files
classes = files.upload()

Saving imagenet_classes.txt to imagenet_classes.txt


In [ ]:
# imagenet의 클래스 분류를 통한 최종결과 확인
with open('/content/imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]
_, index = torch.max(out, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(labels[index[0]], percentage[index[0]].item())

285, Egyptian_cat 52.66728973388672


In [ ]:
# 상위 5개 클래스 보여줌
_, indices = torch.sort(out, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('285, Egyptian_cat', 52.66728973388672),
 ('331, hare', 19.815946578979492),
 ('104, wallaby', 5.0467987060546875),
 ('298, mongoose', 3.409334659576416),
 ('277, red_fox', 3.145502805709839)]